# TP 2: El Dorado

#### Integrantes
- Mirra, Ariel Marcelo.
- Ricci, Juan Gabriel.

#### Docentes
- Olmedo, Agustin.
- Gioia, Alicia.

### Introduccion
Una buena introducccion

### Cositas

In [3]:
# csv aca


1


In [2]:
# plotear ambos por separado
import matplotlib as plt


2


In [ ]:
# la interseccion y toda la wea